In [9]:
import os
import requests
import time
import logging
import json
import pandas as pd
from datetime import datetime
from openpyxl import load_workbook

# ----------------------------
# Constants
# ----------------------------
VALID_YEARS = {str(year) for year in range(2010, 2025)}

STATE_ABBREVIATIONS = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA",
    "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK",
    "OR", "PA", "PR", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]
PREFIXES = ["H", "S", "C"]  # House, Senate, Congress

# ----------------------------
# Logging Configuration
# ----------------------------
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[
        logging.FileHandler("legiscan_api.log"),
        logging.StreamHandler()
    ]
)

# API key and base URL
API_KEY = "c54a2cddb8d0e7b54de012389c3b0fda"
BASE_URL = "https://api.legiscan.com/"

# ----------------------------
# API Cache Setup for Static Data
# ----------------------------
# Since you're retrieving historical data (which is static), cache each API call indefinitely.
CACHE_THRESHOLDS = {
    "getSessionList": float('inf'),
    "getMasterList": float('inf'),
    "getMasterListRaw": float('inf'),
    "getBill": float('inf'),
    "getBillText": float('inf'),
    "getAmendment": float('inf'),
    "getSupplement": float('inf'),
    "getRollcall": float('inf'),
    "getPerson": float('inf'),
    "getSponsoredList": float('inf')
}

# In-memory cache dictionary
# Keys: (operation, frozenset of parameter items)
# Values: dictionary with keys "data" and "timestamp"
api_cache = {}

def cached_api_call(op, params):
    """
    Call the API and cache the response indefinitely.
    """
    key = (op, frozenset(params.items()))
    now = time.time()
    threshold = CACHE_THRESHOLDS.get(op, 0)
    
    if key in api_cache:
        logging.info(f"Using cached response for op {op} with params {params}")
        return api_cache[key]["data"]
    
    # Include API key and op in the parameters
    params.update({"key": API_KEY, "op": op})
    
    while True:
        try:
            response = requests.get(BASE_URL, params=params)
            if response.status_code == 429:
                logging.warning("HTTP 429 Rate limit hit. Sleeping for 60 seconds.")
                time.sleep(60)
                continue
            response.raise_for_status()
            data = response.json()
            if data.get("status") != "OK":
                alert_msg = data.get("alert", {}).get("message", "").lower()
                if "rate" in alert_msg:
                    logging.warning(f"Rate limit alert from API: {alert_msg}. Sleeping for 60 seconds.")
                    time.sleep(60)
                    continue
                logging.error(f"API call failed for op {op} with params {params}: {data}")
                return None
            logging.info(f"Successful API call: op {op}, params {params}")
            api_cache[key] = {"data": data, "timestamp": now}
            return data
        except Exception as e:
            logging.error(f"Exception during API call op {op} with params {params}: {e}")
            logging.warning("Sleeping for 60 seconds before retrying.")
            time.sleep(60)

# ----------------------------
# Excel Appending Helper Function
# ----------------------------
def append_record_to_excel(filename, record):
    """
    Append a single record (dictionary) to an Excel file.
    Nested fields are converted to JSON strings.
    """
    # Convert nested structures to JSON strings if needed
    record_processed = {}
    for key, value in record.items():
        if isinstance(value, (dict, list)):
            record_processed[key] = json.dumps(value)
        else:
            record_processed[key] = value
    df = pd.DataFrame([record_processed])
    
    if not os.path.exists(filename):
        df.to_excel(filename, index=False)
        logging.info(f"Created new Excel file: {filename}")
    else:
        # Append to the existing file
        try:
            with pd.ExcelWriter(filename, mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
                # Load the existing workbook and determine the last row in the sheet
                writer.book = load_workbook(filename)
                sheet_name = writer.book.sheetnames[0]
                writer.sheets = {ws.title: ws for ws in writer.book.worksheets}
                startrow = writer.sheets[sheet_name].max_row
                df.to_excel(writer, index=False, header=False, startrow=startrow)
                logging.info(f"Appended record to Excel file: {filename}")
        except Exception as e:
            logging.error(f"Error appending to Excel file {filename}: {e}")

# ----------------------------
# API Operation Functions Using Cache
# ----------------------------
def get_session_list(state):
    data = cached_api_call("getSessionList", {"state": state})
    if data and "sessions" in data:
        logging.info(f"Retrieved {len(data['sessions'])} sessions for state {state}")
        return data["sessions"]
    logging.warning(f"No session data for state {state}")
    return []

def get_master_list(session_id):
    data = cached_api_call("getMasterList", {"id": session_id})
    if data and "masterlist" in data:
        masterlist = data["masterlist"]
        logging.info(f"Session {session_id}: Retrieved master list with {len(masterlist)} bills")
        return list(masterlist.values())
    logging.warning(f"Session {session_id}: No master list data")
    return []

def get_bill_details(bill_id):
    data = cached_api_call("getBill", {"id": bill_id})
    if data and "bill" in data:
        logging.info(f"Bill {bill_id}: Retrieved detailed information")
        return data["bill"]
    logging.warning(f"Bill {bill_id}: No detailed information retrieved")
    return {}

def get_bill_text(doc_id):
    data = cached_api_call("getBillText", {"id": doc_id})
    if data and "text" in data:
        logging.info(f"Retrieved bill text for doc_id {doc_id}")
        return data["text"]
    logging.warning(f"No bill text for doc_id {doc_id}")
    return {}

def get_amendment(amendment_id):
    data = cached_api_call("getAmendment", {"id": amendment_id})
    if data and "amendment" in data:
        logging.info(f"Retrieved amendment for id {amendment_id}")
        return data["amendment"]
    logging.warning(f"No amendment for id {amendment_id}")
    return {}

def get_supplement(supplement_id):
    data = cached_api_call("getSupplement", {"id": supplement_id})
    if data and "supplement" in data:
        logging.info(f"Retrieved supplement for id {supplement_id}")
        return data["supplement"]
    logging.warning(f"No supplement for id {supplement_id}")
    return {}

def get_roll_call(roll_call_id):
    data = cached_api_call("getRollcall", {"id": roll_call_id})
    if data and "roll_call" in data:
        logging.info(f"Retrieved roll call for id {roll_call_id}")
        return data["roll_call"]
    logging.warning(f"No roll call for id {roll_call_id}")
    return {}

def get_person(people_id):
    data = cached_api_call("getPerson", {"id": people_id})
    if data and "person" in data:
        logging.info(f"Retrieved person data for id {people_id}")
        return data["person"]
    logging.warning(f"No person data for id {people_id}")
    return {}

def get_sponsored_list(people_id):
    data = cached_api_call("getSponsoredList", {"id": people_id})
    if data and "sponsoredbills" in data:
        logging.info(f"Retrieved sponsored bills for person id {people_id}")
        return data["sponsoredbills"]
    logging.warning(f"No sponsored bills for person id {people_id}")
    return {}

# ----------------------------
# Checkpoint Functions
# ----------------------------
CHECKPOINT_FILE = "checkpoint.json"

def save_checkpoint(year, state, session_idx, bill_idx):
    checkpoint = {
        "year": year,
        "state": state,
        "session_idx": session_idx,
        "bill_idx": bill_idx
    }
    with open(CHECKPOINT_FILE, "w") as f:
        json.dump(checkpoint, f)
    logging.info(f"Checkpoint saved: {checkpoint}")

def load_checkpoint():
    try:
        with open(CHECKPOINT_FILE, "r") as f:
            checkpoint = json.load(f)
        logging.info(f"Checkpoint loaded: {checkpoint}")
        return checkpoint
    except Exception:
        logging.info("No checkpoint found, starting fresh.")
        return None

# ----------------------------
# Process a Single State for a Given Year
# ----------------------------
def process_state_year(year, state, checkpoint=None):
    filename = f"legiscan_{state}_{year}.xlsx"
    logging.info(f"--- Starting processing for {state} for year {year} ---")
    sessions = get_session_list(state)
    # Filter sessions by year using session_name or year boundaries
    year_sessions = [s for s in sessions if str(year) in str(s.get("session_name", "")) or 
                     (s.get("year_start") and s.get("year_start") <= year <= s.get("year_end", year))]
    logging.info(f"State {state}: Found {len(year_sessions)} sessions for year {year}")
    
    for sess_idx, session in enumerate(year_sessions):
        # Resume checkpoint if applicable
        if checkpoint and checkpoint.get("year") == year and checkpoint.get("state") == state:
            if sess_idx < checkpoint.get("session_idx", 0):
                continue
            bill_resume = checkpoint.get("bill_idx", 0) if sess_idx == checkpoint.get("session_idx", 0) else 0
        else:
            bill_resume = 0

        session_id = session.get("session_id")
        logging.info(f"State {state}, Session {session_id}: Retrieving master list")
        master_list = get_master_list(session_id)
        for bill_idx, bill_summary in enumerate(master_list):
            if bill_idx < bill_resume:
                continue

            bill_id = bill_summary.get("bill_id")
            bill_details = get_bill_details(bill_id)
            if not bill_details:
                logging.warning(f"Skipping bill {bill_id} due to missing details.")
                continue

            # Combine summary and detailed info into a single record
            record = {**bill_summary, **bill_details}
            record["state"] = state
            record["year"] = year

            # Retrieve full bill text if available
            if bill_details.get("doc_id"):
                record["bill_text"] = get_bill_text(bill_details["doc_id"])
            else:
                record["bill_text"] = {}

            # Retrieve amendments (if referenced)
            amendments = bill_details.get("amendments", [])
            record["amendments"] = [get_amendment(amendment.get("amendment_id")) 
                                    for amendment in amendments if amendment.get("amendment_id")]

            # Retrieve supplements (if referenced)
            supplements = bill_details.get("supplements", [])
            record["supplements"] = [get_supplement(supplement.get("supplement_id"))
                                     for supplement in supplements if supplement.get("supplement_id")]

            # Retrieve roll call details if available
            if bill_details.get("roll_call_id"):
                record["roll_call"] = get_roll_call(bill_details["roll_call_id"])
            else:
                record["roll_call"] = {}

            # Retrieve detailed sponsor info and sponsored bills
            sponsors = bill_details.get("sponsors", [])
            detailed_sponsors = []
            for sponsor in sponsors:
                people_id = sponsor.get("people_id")
                if people_id:
                    person_data = get_person(people_id)
                    sponsored_bills = get_sponsored_list(people_id)
                    sponsor_record = {**sponsor, "person_details": person_data, "sponsored_bills": sponsored_bills}
                    detailed_sponsors.append(sponsor_record)
            record["sponsors"] = detailed_sponsors

            # Append the processed record to the Excel file immediately
            append_record_to_excel(filename, record)

            logging.info(f"Stored bill {bill_id} - {record.get('number')} - {record.get('title')}")
            # Save checkpoint after each bill processed
            save_checkpoint(year, state, sess_idx, bill_idx)
            
    logging.info(f"--- Completed processing for {state} for year {year} ---")

# ----------------------------
# Main Function: Process Each State per Year
# ----------------------------
def main():
    checkpoint = load_checkpoint()
    for year in range(2024, 2009, -1):
        for state in STATE_ABBREVIATIONS:
            logging.info(f"=== Processing {state} for year {year} ===")
            process_state_year(year, state, checkpoint=checkpoint)
            # Reset checkpoint for next state/year combination
            checkpoint = None

if __name__ == "__main__":
    main()

2025-03-12 16:44:37,687 [INFO] Checkpoint loaded: {'year': 2024, 'state': 'AZ', 'session_idx': 0, 'bill_idx': 994}
2025-03-12 16:44:37,689 [INFO] === Processing AL for year 2024 ===
2025-03-12 16:44:37,690 [INFO] --- Starting processing for AL for year 2024 ---
2025-03-12 16:44:37,940 [ERROR] API call failed for op getSessionList with params {'state': 'AL', 'key': 'c54a2cddb8d0e7b54de012389c3b0fda', 'op': 'getSessionList'}: {'status': 'ERROR', 'alert': {'message': 'API key has exceeded maximum query count for March 2025 (30,082 of 30,000); limit resets April 1st [Creating additional keys to bypass this limit will result in suspended access]', 'contact': {'email': 'api@legiscan.com', 'phone': '800-618-2750'}}}
2025-03-12 16:44:37,943 [WARNING] No session data for state AL
2025-03-12 16:44:37,943 [INFO] State AL: Found 0 sessions for year 2024
2025-03-12 16:44:37,944 [INFO] --- Completed processing for AL for year 2024 ---
2025-03-12 16:44:37,945 [INFO] === Processing AK for year 2024 ==

KeyboardInterrupt: 